Questo notebook esegue il training del modello CUT per adattare il dominio **Reale** al dominio **Sintetico**.

- **Repository:** [AntonioRosano/benchmarking-generative-models-for-domain-adaptation](https://github.com/AntonioRosano/benchmarking-generative-models-for-domain-adaptation)

- **Prerequisiti:**
Avere il Dataset delle immagini caricato (es. `EGO-CH-OBJ-SEG`).

In [ ]:
import os
import shutil


%cd /kaggle/working
if os.path.exists("benchmarking-generative-models-for-domain-adaptation"):
    shutil.rmtree("benchmarking-generative-models-for-domain-adaptation")


print("Clono la repository...")
!git clone --recursive https://github.com/AntonioRosano/benchmarking-generative-models-for-domain-adaptation.git
%cd benchmarking-generative-models-for-domain-adaptation


print("Installo librerie...")
!pip install -q dominate visdom

In [ ]:
# Patch per il bug noto di argparse in cut_model.py
target_file = "./models/cut/models/cut_model.py"

if os.path.exists(target_file):
    with open(target_file, "r") as f:
        content = f.read()
    
    # Cambia choices='(...)' in choices=['...']
    new_content = content.replace("choices='(CUT, cut, FastCUT, fastcut)'", "choices=['CUT', 'cut', 'FastCUT', 'fastcut']")
    
    with open(target_file, "w") as f:
        f.write(new_content)
    print("Patch applicata correttamente a cut_model.py")
else:
    print("Attenzione: File cut_model.py non trovato. Controlla il percorso.")


In [ ]:
import os
import shutil


REAL_SOURCE = "/kaggle/input/ego-ch/EGO-CH-OBJ-SEG/real/train/frames"
SYN_SOURCE = "/kaggle/input/ego-ch/EGO-CH-OBJ-SEG/synthetic/train/frames"


WORK_DIR = "/kaggle/working/benchmarking-generative-models-for-domain-adaptation/models/cut"
DEST_DATASET = os.path.join(WORK_DIR, "datasets", "ego_adaptation")



if not os.path.exists(WORK_DIR):
    print(f"Errore: Non trovo la cartella {WORK_DIR}. Hai clonato la repo?")
else:
    %cd {WORK_DIR}

    if os.path.exists(DEST_DATASET):
        print("Pulisco la cartella dataset vecchia...")
        shutil.rmtree(DEST_DATASET)

    os.makedirs(os.path.join(DEST_DATASET, "trainA"), exist_ok=True)
    os.makedirs(os.path.join(DEST_DATASET, "trainB"), exist_ok=True)


    print(f"Copio immagini REALI da: {REAL_SOURCE} ...")
    !cp "{REAL_SOURCE}"/* "{DEST_DATASET}/trainA/"
    
    print(f"Copio immagini SINTETICHE da: {SYN_SOURCE} ...")
    !cp "{SYN_SOURCE}"/* "{DEST_DATASET}/trainB/"


    count_A = len(os.listdir(f"{DEST_DATASET}/trainA"))
    count_B = len(os.listdir(f"{DEST_DATASET}/trainB"))
    print(f"\nSTATO: trainA={count_A}, trainB={count_B}")

In [ ]:
!python train.py \
          --dataroot ./datasets/ego_adaptation \
          --name ego_cut_kaggle \
          --CUT_mode CUT \
          --n_epochs 20 \
          --n_epochs_decay 20 \
          --batch_size 4 \
          --num_threads 4 \
          --display_id -1 \
          --save_epoch_freq 5 \
          --print_freq 100

In [ ]:
import matplotlib.pyplot as plt
import re
import os

log_path = "./checkpoints/ego_cut_kaggle/loss_log.txt"
epochs, G_GAN, NCE = [], [], []

if os.path.exists(log_path):
    print("Generazione grafico loss...")
    with open(log_path, 'r') as f:
        for line in f:
            if "epoch:" in line and "G_GAN:" in line:
                ep = int(re.search(r'epoch:\s*(\d+)', line).group(1))
                if len(epochs) == 0 or ep != epochs[-1]:
                    epochs.append(ep)
                    G_GAN.append(float(re.search(r'G_GAN:\s*([\d\.]+)', line).group(1)))
                    NCE.append(float(re.search(r'NCE:\s*([\d\.]+)', line).group(1)))

    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, G_GAN, label='G_GAN')
    plt.title("Generator Adversarial Loss")
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.subplot(1, 2, 2)
    plt.plot(epochs, NCE, color='orange', label='NCE')
    plt.title("PatchNCE Loss (Content)")
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    save_path = os.path.join(os.path.dirname(log_path), "loss_plot.png")
    plt.savefig(save_path)
    
    plt.show()
else:
    print("Nessun log trovato. Il training è partito?")

In [ ]:
%cd /kaggle/working/benchmarking-generative-models-for-domain-adaptation/models/cut

print("Zippo i checkpoint...")
!zip -r checkpoints_epoch40.zip ./checkpoints/ego_cut_kaggle/

print("Zip pronto.")

#Al termine, scaricare lo zip, estrarlo e caricarlo come dataset nel notebook "Test_CUT_Kaggle"